In [1]:
import glob
import re
import bs4
import cltk
import nltk

import numpy as np
from collections import defaultdict, Counter
from nltk.tokenize.punkt import PunktLanguageVars

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import ward, dendrogram   

from sklearn.feature_extraction.text import TfidfVectorizer
from cltk.stop.latin.stops import STOPS_LIST

import itertools

stop_words = STOPS_LIST.extend(['quod', 'vel', 'sunt', 'hoc', 'vero', 'sit', 'sol', 'que', 'esse'])

%matplotlib inline

In [2]:
from cltk.corpus.utils.importer import CorpusImporter

corpus_importer = CorpusImporter('latin')
corpus_importer.import_corpus('latin_text_latin_library')
corpus_importer.import_corpus('latin_pos_lemmata_cltk')
corpus_importer.import_corpus('latin_models_cltk')

INFO:CLTK:Pulling latest 'latin_text_latin_library' from 'https://github.com/cltk/latin_text_latin_library.git'.
INFO:CLTK:Pulling latest 'latin_pos_lemmata_cltk' from 'https://github.com/cltk/latin_pos_lemmata_cltk.git'.
INFO:CLTK:Pulling latest 'latin_models_cltk' from 'https://github.com/cltk/latin_models_cltk.git'.


In [3]:
# This corpus comes from the Thesaurus Musicarum Latinarum.

corpus_files = glob.glob('corpus/html/*.html')

In [4]:
soups = [bs4.BeautifulSoup(open(file), 'lxml') for file in corpus_files]

In [5]:
def process_soup(soup):
    "Takes a bs4 BeautifulSoup object and returns a dict with document content and metadata."
    corpus_item = {}
    
    header = soup.find('p')

    original = " ".join([n for n in soup.findAll(text=True)])
    
    body = "".join([p.text for p in soup.findAll('p')[1:]])
    body = remove_newlines(body)
    body = remove_doublespace(body)
    body = remove_editorial_apparatus(body)
    body = join_hyphens(body)
    
    century_re = re.compile('.*www.chmtl.indiana.edu\/tml\/(\w+\W*\w+)\/.*')
    
    try:
        comments = "".join(soup.findAll(text=lambda text:isinstance(text, bs4.Comment)))
        century = century_re.match(comments).groups()[0]
        corpus_item['century'] = century
    except:
        corpus_item['century'] = 'nd'

    corpus_item['body'] = body.strip()
    corpus_item['id'] = soup.title.text.split(' ')[0]
    
    match = re.search(r"Author:\s(.*)", original)
    
    if match:
        result = match.group(1)
    else:
        result = ""
    
    corpus_item['author'] = result
    
    match = re.search(r"Title:\s(.*)", original)
    
    if match:
        result = match.group(1)
    else:
        result = ""
        
    corpus_item['title'] = result
    
    return corpus_item

def remove_editorial_apparatus(text):
    "Removes all text enclosed in square brackets."
    pattern = re.compile('\[.+?\]')
    return pattern.sub('', text)

def remove_newlines(text):
    "Replaces newlines with spaces."
    return text.replace('\n', ' ')

def remove_doublespace(text):
    "Removes any doublespaces."
    return text.replace('  ', ' ')

def join_hyphens(text):
    "Joins hyphens used at ends of lines."
    return text.replace('-\n', '')

def my_tokenize(document):
    "Tokenizes a document, represented as a string using Punkt."
    p = PunktLanguageVars()
    tokens = p.word_tokenize(document)
    tokens = [x.lower() for x in tokens if x not in (',', ';', '.', "'", '"',':',')','(', '|' , '||' )]
    return tokens

In [6]:
corpus = []

for soup in soups:
    try:
        corpus_item = process_soup(soup)    
        corpus.append(corpus_item)
    except:
        continue

In [7]:
len(corpus)

807

In [8]:
# Write texts stripped of boilerplate for use externally 

import unidecode

for item in corpus:
    with open('./corpus/body/{}.txt'.format(item['id']), 'w') as f:
        f.write(unidecode.unidecode(item['body']))

In [9]:
def prepare_id_info_map(corpus):
    id_info_map = {}
    for item in corpus:
        item_copy = item.copy()
        del item_copy['body']
        id_info_map[item['id']] = item_copy
    return id_info_map

In [10]:
id_info_map = prepare_id_info_map(corpus)

In [11]:
id_info_map['AARIH1']

{'author': 'Aaron, Petrus',
 'century': '16th',
 'id': 'AARIH1',
 'title': 'De institutione harmonica, liber primus'}

In [12]:
def filter_stopwords(tokens):
    "Filters stopwords from list of tokens."
    return [token for token in tokens if token not in STOPS_LIST]

def filter_shortwords(tokens, short_size=2):
    "Filters tokens of size greater than short_size from list of tokens."
    return [token for token in tokens if len(token) > short_size]

In [13]:
from cltk.stem.lemma import LemmaReplacer

lemmatizer = LemmaReplacer('latin')

In [14]:
def prep(document):
    "A convenience function which applies a series of operations to a document represented in a string."
    tokenized = my_tokenize(document)
    stopped = filter_stopwords(tokenized)
    shorted = filter_shortwords(stopped)
    lemmatized = lemmatizer.lemmatize(shorted)
    done = lemmatized
    return done

## Topic Modeling with `gensim`

In [ ]:
import gensim

In [ ]:
documents = [prep(doc['body']) for doc in corpus]

In [ ]:
dictionary = gensim.corpora.Dictionary(documents)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
len(bow_corpus)

In [ ]:
gensim.corpora.MmCorpus.serialize('tml.mm', bow_corpus)

In [ ]:
bow_corpus = gensim.corpora.MmCorpus('tml.mm')

In [ ]:
tfidf = gensim.models.TfidfModel(bow_corpus, normalize=True)
tfidf_corpus = tfidf[bow_corpus]

In [ ]:
lsi = gensim.models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=150)

In [ ]:
lsi.print_topics(100)[60:90]

## Focusing on a subset

This is some code to restrict sample to a certain set of parameters

TODO: Make a proper query interface for corpus entries

In [ ]:
q_century = '17th'
q_min_size = 50
q_max_size = 10000000

sample = [treatise for treatise in corpus\
          if treatise['century'] == q_century\
          and len(treatise['body']) < q_max_size\
          and len(treatise['body']) > q_min_size]

bodys = [treatise['body'] for treatise in sample]
ids = [treatise['id'] for treatise in sample]
titles = [treatise.get('title', 'nt')[:15] for treatise in sample]
authors = [treatise.get('author', 'na')[:15] for treatise in sample]

len(sample) , len(ids), len(titles)

In [ ]:
lengths = [len(treatise['body']) for treatise in sample]
plt.hist(lengths, bins=12)
plt.title('Histogram of text lengths (characters) in 13C treatises.')
plt.show()

## Text Similarity - Ward clustering in tf-idf space

In [ ]:
documents = [doc['body'] for doc in corpus]

tfidf = TfidfVectorizer(tokenizer=prep, stop_words=stop_words)
tfs = tfidf.fit_transform(documents)

In [ ]:
# bodys is the sample only from the 13th century 

tfs = tfidf.transform(bodys)
tfs.shape

dist = 1 - cosine_similarity(tfs)

In [ ]:
linkage_matrix = ward(dist)

fig, ax = plt.subplots(figsize=(5, 15)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=authors);

plt.tick_params(axis='x',          
    which='both',      
    bottom='off',      
    top='off',         
    labelbottom='off')

plt.tight_layout() 

## Text reuse - Viral Texts

Reimplementation of the Viral Texts methodology

viraltexts.org

Note that this is done at the "section" level - will need to "sectionize" documents first.

Part I. Identify document pair candidates

1. Pick out a subset of all the texts for testing purposes
2. Sectionalize
3. ~~Make a hashmap id -> tokenized text~~
4. Make a hashmap id -> shingled text
5. For each n-gram, create a list of (d_i, p_i) - document ID and position ID
6. Exclude singleton n-grams
7. Extract candidate pairs
8. Filter (e.g. suppress n-grams that generate more than some fixed number of pairs)
9. Rank

Part II. Local document alignment

1. Do Smith-Waterman on top pairs

In [15]:
import random

N = 5

def zipngram(document_tokens,n=N):
    return zip(*[document_tokens[i:] for i in range(n)])

In [16]:
def prepare_ngram_index(id_shingled_map):
    ngram_index = defaultdict(list)
    
    for _id, shingled_rep in id_shingled_map.items():
        for position, n_gram in enumerate(shingled_rep):
            ngram_index[n_gram].append((_id, position))
    
    return ngram_index

def filter_singletons(ngram_index):
    # Naive implementation not advised in article
    # TODO: Implement nice two-pass hash fxn. method
    filtered_ngram_index = {}
    
    for ngram, locations in ngram_index.items():
        if len(locations) > 1:
            filtered_ngram_index[ngram] = locations
    
    return filtered_ngram_index

def prepare_id_tokenized_map(corpus):
    id_tokenized_map = {}
    
    for text in corpus:
        id_tokenized_map[text['id']] = my_tokenize(text['body'])
        
    return id_tokenized_map

def prepare_id_body_map(corpus):
    id_body_map = {}
    
    for text in corpus:
        id_body_map[text['id']] = text['body']
            
    return id_body_map 

def prepare_id_shingled_map(corpus): 
    id_shingled_map = {}
    
    for text in corpus:
        tokenized_body = my_tokenize(text['body'])
        id_shingled_map[text['id']] = [ngram for ngram in zipngram(tokenized_body, N)]
    
    return id_shingled_map

def prepare_pair_ngrams_map(ngram_index):
    pair_ngrams_map = defaultdict(list)
    
    for ngram, locations in ngram_index.items():
        documents = list(set([location[0] for location in locations]))
        for p in itertools.combinations(documents, 2):
            if p[0][:3] != p[1][:3]:
                pair_ngrams_map[p].append(ngram)
    
    return pair_ngrams_map

In [17]:
corpus[0].keys()

dict_keys(['title', 'century', 'author', 'body', 'id'])

In [18]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]
    
def sectionalize_corpus(corpus, section_length=50):
    sectionalized_corpus = []
    for item in corpus:
        words = item['body'].split(' ')
        for index, chunk in enumerate(chunks(words, section_length)):
            old_id = item.get('id')
            new_item = item.copy()
            new_item['id'] = old_id + '_{}'.format(index)
            new_item['body'] = ' '.join(chunk)
            sectionalized_corpus.append(new_item)
    return sectionalized_corpus        

In [19]:
sc = sectionalize_corpus(corpus)

In [20]:
sc[100]

{'author': 'Jacobus Leodiensis',
 'body': 'b mollem secundum ut de F; sed re per  durum secundum ut de G. Et consimili modo intelligendum est de distinctis vocibus in aliis positis clavibus. Ex his etiam videri potest quare in tribus primis litteris et in ultima sunt sole voces, quare in quarta due, in quinta due,',
 'century': '14th',
 'id': 'JACSM6B_100',
 'title': 'Speculum musicae, Liber sextus'}

In [21]:
%time id_body_map = prepare_id_body_map(sc)

CPU times: user 60.4 ms, sys: 8.32 ms, total: 68.8 ms
Wall time: 74.6 ms


In [22]:
%time id_tokenized_map = prepare_id_tokenized_map(sc)

CPU times: user 15.4 s, sys: 174 ms, total: 15.5 s
Wall time: 15.6 s


In [23]:
%time id_shingled_map = prepare_id_shingled_map(sc)

CPU times: user 17.2 s, sys: 396 ms, total: 17.6 s
Wall time: 17.6 s


In [24]:
%time idx = prepare_ngram_index(id_shingled_map)

CPU times: user 15.9 s, sys: 14.5 s, total: 30.4 s
Wall time: 1min 1s


In [25]:
%time filtered_idx = filter_singletons(idx)

CPU times: user 2.38 s, sys: 1.94 s, total: 4.32 s
Wall time: 10.7 s


In [26]:
assert len(idx) > len(filtered_idx)

Continuing to search for candidate document pairs, let's go on an make a dict that has pairs of documents as its keys and a list of ngrams that each share is its values.

In [27]:
%time pair_ngrams_map = prepare_pair_ngrams_map(filtered_idx)

CPU times: user 4.59 s, sys: 1.06 s, total: 5.66 s
Wall time: 7.91 s


In [28]:
def p_n_map_ngrams_freq(pair_ngrams_map):
    freqs = Counter()
    for pair, ngrams in pair_ngrams_map.items():
        freqs[pair] += len(ngrams)
    return freqs

In [29]:
freq = p_n_map_ngrams_freq(pair_ngrams_map)
len(freq)

486598

In [66]:
pairs_to_align = [f[0] for f in freq.most_common(100000)]
len(pairs_to_align)

100000

Do Smith-Waterman using multiprocessing

In [31]:
from Bio import pairwise2
import multiprocessing
import subprocess
import align
import numpy

In [ ]:
%%time 

def best_alignment_by_id(id_tuple):
    text_a = id_body_map[id_tuple[0]]
    text_b = id_body_map[id_tuple[1]]
    a = pairwise2.align.localms(text_a, text_b, 2, -1, -.5, -.1)
    return (id_tuple, a[0])

pool = multiprocessing.Pool(processes=2)
pool_outputs = pool.map(best_alignment_by_id, pairs_to_align)
pool.close() 
pool.join() 

print(len(pool_outputs))

This implementation is embarassingly slow. This seems legit:

https://github.com/noporpoise/seq-align

but it doesn't have a Python API (yet...)

In [32]:
%%time

def work(cmd):
    return subprocess.call(cmd, shell=False)

def invoke_seq_align(id_tuple):
    text_a = id_body_map[id_tuple[0]]
    text_b = id_body_map[id_tuple[1]]
    args = ['/home/eamonn/Downloads/seq-align/bin/smith_waterman', text_a, text_b]
    return work(args)

count = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=count)
print(pool.map(invoke_seq_align, pairs_to_align))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CPU times: user 10.4 ms, sys: 122 ms, total: 133 ms
Wall time: 13 s


In [68]:
S = -numpy.ones((256, 256)) + 2 * numpy.identity(256)
S = S.astype(numpy.int16)

In [69]:
%%time

f = open('out.put', 'w')

def invoke_align(id_tuple):
    text_a = id_body_map[id_tuple[0]]
    text_b = id_body_map[id_tuple[1]]
    s1 = align.string_to_alignment(text_a)
    s2 = align.string_to_alignment(text_b)
    (s, a1, a2) = align.align(s1, s2, -2, -2, S, local=True)
    a1s = align.alignment_to_string(a1)
    a2s = align.alignment_to_string(a2)
    f.write('{}\t{}\t{}\t{}\n'.format(id_tuple, a1s, a2s, s))
    
    return 0

count = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=2)
print(len(pool.map(invoke_align, pairs_to_align)))

100000
CPU times: user 394 ms, sys: 568 ms, total: 961 ms
Wall time: 1min 40s


---

## Extra stuff


### Create a borrowing network

Create a digraph with weighted edges from the pair-ngram-map where the weight is given by the number of shared n-grams.

In [ ]:
def p_n_map_to_weighted_edges(pair_ngrams_map, min_ngrams=50, max_ngrams=5000):
    edges = []
    for pair, ngrams in pair_ngrams_map.items():
        if min_ngrams < len(ngrams) < max_ngrams:
            edges.append("{},{},{}\n".format(pair[0], pair[1], len(ngrams)))
    
    return edges

In [ ]:
edges = p_n_map_to_weighted_edges(pair_ngrams_map)

In [ ]:
def write_edges_to_file(edges, file=open('graph.txt', 'w')):
    file.write('source,target,weight\n')
    file.writelines(edges)
    file.close()

In [ ]:
write_edges_to_file(edges)

# Text reuse - Naive n-gram recurrence heatmap

This was the original effort to visualize text reuse. It would be nice to have this interactive, with a hover system that showed:

1. The ngram that is being shared
2. Sources for each n-gram (reference, link)
3. Other statistics about the n-gram

Note that the tokenizer here is really dumb; and we don't do any lemmatizing etc. This is because we're just interested here in literal text reuse.

Could reuse the indexes generated above to acheive the same ends.

1. Convert each document to a list of n-grams (say n=3)
2. Convert the remainder of the corpus to a list of n-grams (same n), and that into a Counter
3. For each document, iterate through the list of n-grams
4. If the n-gram is appears in the rest of the corpus, make a note of that and how many times it appears
5. End up with a "heatmap" of reuse

In [ ]:
from nltk.util import ngrams

import itertools

N = 3

sample = random.sample(corpus, 25)
bodys = [item['body'] for item in sample]
ids = [item['id'] for item in sample]

corpus_tokens_list = [my_tokenize(document) for document in bodys]

def zipngram(document_tokens,n=N):
    return zip(*[document_tokens[i:] for i in range(n)])

def generate_slice(index, document, sample):
    # Document being studied for text reuse
    document = document
    # Make a list of the other documents
    others = [d for j, d in enumerate(sample) if j != index]

    # Tokenize
    document_tokens = my_tokenize(document)
    others_tokens_list = corpus_tokens_list

    # Get n-grams
    document_ngrams_generator = zipngram(document_tokens, N)
    document_ngrams = [x for x in document_ngrams_generator]
    others_ngrams_list = [zipngram(other_tokens, N) for other_tokens in others_tokens_list]

    # Create counter implemented with defaultdict for remainders
    flat = itertools.chain(*others_ngrams_list)
    o_counter = Counter(flat)

    heatslice = [o_counter.get(ngram, 0) for ngram in document_ngrams]
    return heatslice, document_ngrams

def construct_heatmap():
    for index, document in enumerate(bodys):
        a_slice = generate_slice(index, document, bodys)
        heatmap.append(a_slice[0])
        documents_ngrams.append(a_slice[1])

def plot_heatmap2():
    # Use subplots and contourf to create heatmap bands (per document)
    f, axarr = plt.subplots(len(heatmap), figsize=(15, 20))
    for index, data in enumerate(heatmap):
        h = np.array(data)
        x = np.empty([2,h.shape[0]])
        x[:,:] = h
        axarr[index].contourf(x)
    f.show()

def plot_heatmap():
    # First go which just puts everything on the one heatmap
    length = len(sorted(heatmap,key=len, reverse=True)[0])
    hm = np.array([xi+[0]*(length-len(xi)) for xi in heatmap])
    fig, ax = plt.subplots(figsize=(15, 20))
    ax.set_yticks(np.arange(hm.shape[0])+0.5, minor=False)
    ax.set_yticklabels(ids)
    ax.pcolormesh(hm)
    fig.show()

In [ ]:
heatmap = []
documents_ngrams = []

%matplotlib inline
construct_heatmap()

In [ ]:
plot_heatmap2()

----

## More stuff about the indexes compiled earlier 

Let's have a look inside this index, shall we?

In [ ]:
def ngram_index_freq(index, blacklist=[]):
    freqs = Counter()
    for ngram, locations in index.items():
        if not bool(set(ngram) & set(blacklist)):
            freqs[ngram] += len(locations)
    return freqs

freq = ngram_index_freq(filtered_idx)
freq.most_common(20)

There's a lot of scale segments... We could do a couple of things about this, I suppose. Let's find out the most common tokens in the dataset and exclude any ngrams from this list which contain a token that appears close to the top of that list.

In [ ]:
def count_all_tokens(id_tokenized_map):
    all_tokens = []
    for _id, tokens in id_tokenized_map.items():
        all_tokens.extend(tokens)
    return Counter(all_tokens)

token_counter = count_all_tokens(id_tokenized_map)

def top_n_tokens(id_tokenized_map, n):
    token_counter = count_all_tokens(id_tokenized_map)
    return [t[0] for t in token_counter.most_common(n)]

top_100_tokens = top_n_tokens(id_tokenized_map, 100)

freq = ngram_index_freq(filtered_idx, blacklist=top_100_tokens)
freq.most_common(20)

A bit better this time. Let's check out the provenance of the phrase:

> acuti soni gravisque mixtura suaviter

which appears 26 times in the corpus.

In [ ]:
locations = filtered_idx[('acuti', 'soni', 'gravisque', 'mixtura', 'suaviter')]
for _id, position in locations:
    info = id_info_map[_id]
    print(_id, info['author'][:15], info['title'][:15], position, info['century'])

How about this for a research question?

> What treatises does Descartes allude to/paraphrase in his *Compendium*?

1. Use id_shingled_map to get the shingled version of Descartes' text, 'DESCOM'
2. Look up each n-gram in the filtered_idx for concordances
3. Filter out disinteresting ones
4. Print out the output

In [ ]:
query_id = 'DESCOM'
query_shingled = [ngram for ngram in id_shingled_map[query_id]]

len(query_shingled)

In [ ]:
response = []

for query_ngram in query_shingled:
    locations = filtered_idx.get(query_ngram, None)
    if locations is not None:
        response.append((query_ngram, locations))